<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a>
</p>
<p align="center">
<a href="https://discord.gg/AMApC2UzVY"><img alt="Discord" src="https://img.shields.io/badge/discord-chat-purple?color=%235765F2&label=discord&logo=discord"></a>
<a href="https://twitter.com/vlmrun"><img alt="Twitter Follow" src="https://img.shields.io/twitter/follow/vlmrun.svg?style=social&logo=twitter"></a>
</p>
</div>

Welcome to **[VLM Run Cookbooks](https://github.com/vlm-run/vlmrun-cookbook)**, a comprehensive collection of examples and notebooks demonstrating the power of structured visual understanding using the [VLM Run Platform](https://app.vlm.run).


## Image Captioning & Tagging with VLM Run (Node.js)

This notebook demonstrates how to use VLM Run's Agent API to generate detailed captions and tags for images using **Node.js/TypeScript**. Image captioning is perfect for:
- **Accessibility**: Creating alt-text descriptions for images
- **Content Management**: Automatically organizing and cataloging image libraries
- **SEO**: Generating image descriptions for better search engine optimization
- **Automated Analysis**: Building workflows that understand visual content

We'll use VLM Run's vision-language models to generate comprehensive, contextual captions and extract relevant tags from various images.


### Environment Setup

To get started, install the required packages and sign up for an API key on the [VLM Run App](https://app.vlm.run).
- Store the VLM Run API key under the `VLMRUN_API_KEY` environment variable.


## Prerequisites

* Node.js 18+
* VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
* Deno or tslab kernel for running TypeScript in Jupyter


## Setup

First, let's install the required packages:


In [ ]:
// If using Deno kernel, install dependencies via npm specifiers
// For tslab, run: npm install openai zod zod-to-json-schema in your project directory


In [2]:
import OpenAI from "openai";
import { z } from "zod";
import { zodToJsonSchema } from "npm:zod-to-json-schema";


In [ ]:
// Get API key from environment variable
const VLMRUN_API_KEY = Deno.env.get("VLMRUN_API_KEY");

if (!VLMRUN_API_KEY) {
    throw new Error("Please set the VLMRUN_API_KEY environment variable");
}

console.log("✓ API Key loaded successfully");


✓ API Key loaded successfully


Let's initialize the OpenAI client with VLM Run's Agent API endpoint


In [7]:
// Initialize the OpenAI client with VLM Run's Agent API
const client = new OpenAI({
    baseURL: "https://agent.vlm.run/v1/openai",
    apiKey: VLMRUN_API_KEY
});

console.log("✓ OpenAI client initialized with VLM Run Agent API");


✓ OpenAI client initialized with VLM Run Agent API


## Example 1: Simple Image Captioning

Let's start with a simple example - generating a caption for a single image.


In [8]:
// Example image URL
const imageUrl = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg";

// Display the image URL (in Jupyter, you can use display functions)
console.log("Image URL:", imageUrl);
console.log("\n📷 View the image:", imageUrl);


Image URL: https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg

📷 View the image: https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg


Now let's generate a caption for this image:


In [9]:
// Define a helper function to generate captions
async function generateCaption(
    imageUrl: string, 
    prompt: string = "Generate a detailed caption for this image"
): Promise<string> {
    /**
     * Generate a caption for an image using VLM Run's Agent API.
     */
    const response = await client.chat.completions.create({
        model: "vlm-agent-1",
        messages: [
            {
                role: "user",
                content: [
                    { type: "text", text: prompt },
                    { type: "image_url", image_url: { url: imageUrl } }
                ]
            }
        ]
    });
    
    return response.choices[0].message.content || "";
}

// Generate caption for the image
const caption = await generateCaption(imageUrl);
console.log(`Caption: ${caption}`);


Caption: This eye-level shot captures a bustling city street scene, likely in New York City, with towering commercial buildings lining both sides. A diverse group of pedestrians crosses a marked crosswalk, including an adult man with a beard and light blue shirt leading a woman, a small child, and another child in a stroller. Another man with a backpack walks past a bright yellow taxi cab in the mid-ground. The street is filled with various vehicles, including multiple yellow taxis and other cars, reflecting the dynamic urban environment. Storefronts and street signs add to the lively city atmosphere.


## Example 2: Structured Output with Zod Schema

For production applications, you might want structured JSON output. Let's use Zod schemas (Node.js equivalent of Python's Pydantic) with OpenAI's structured outputs.


In [10]:
// Define the schema using Zod
const ImageCaptionSchema = z.object({
    caption: z.string().describe("Detailed caption of the scene"),
    tags: z.array(z.string()).describe("Tags that describe the image"),
    primary_objects: z.array(z.string()).describe("Main objects visible in the image"),
    scene_type: z.string().describe("Type of scene (e.g., indoor, outdoor, street, nature)")
});

// Type inference from schema
type ImageCaption = z.infer<typeof ImageCaptionSchema>;

// Convert Zod schema to JSON Schema for API
const jsonSchema = zodToJsonSchema(ImageCaptionSchema, "ImageCaption");
console.log("JSON Schema:", JSON.stringify(jsonSchema, null, 2));


JSON Schema: {
  "$ref": "#/definitions/ImageCaption",
  "definitions": {
    "ImageCaption": {
      "type": "object",
      "properties": {
        "caption": {
          "type": "string",
          "description": "Detailed caption of the scene"
        },
        "tags": {
          "type": "array",
          "items": {
            "type": "string"
          },
          "description": "Tags that describe the image"
        },
        "primary_objects": {
          "type": "array",
          "items": {
            "type": "string"
          },
          "description": "Main objects visible in the image"
        },
        "scene_type": {
          "type": "string",
          "description": "Type of scene (e.g., indoor, outdoor, street, nature)"
        }
      },
      "required": [
        "caption",
        "tags",
        "primary_objects",
        "scene_type"
      ],
      "additionalProperties": false
    }
  },
  "$schema": "http://json-schema.org/draft-07/schema#"
}


In [15]:
async function generateStructuredCaption(imageUrl: string): Promise<ImageCaption> {
    /**
     * Generate a structured caption using Zod schemas and JSON Schema.
     */
    const response = await client.chat.completions.create({
        model: "vlm-agent-1",
        messages: [
            {
                role: "user",
                content: [
                    { 
                        type: "text", 
                        text: "Analyze this image and provide a detailed caption, relevant tags, primary objects, and scene type." 
                    },
                    { 
                        type: "image_url", 
                        image_url: { url: imageUrl, detail: "auto" } 
                    }
                ]
            }
        ],
        // Use VLM Run's format - schema at top level, not nested under json_schema
        response_format: { 
            type: "json_schema", 
            schema: zodToJsonSchema(ImageCaptionSchema)
        } as any  // Type assertion needed since this differs from OpenAI's type
    });
    
    // Print the raw JSON response
    const rawContent = response.choices[0].message.content || "{}";
    console.log("Raw JSON Response:");
    console.log(rawContent);
    console.log();
    
    // Parse and validate the response
    const parsed = JSON.parse(rawContent);
    return ImageCaptionSchema.parse(parsed);
}

// Generate structured caption
const structuredResult = await generateStructuredCaption(imageUrl);

console.log("Validated Object:");
console.log(structuredResult);
console.log();
console.log("Formatted Output:");
console.log(`Caption: ${structuredResult.caption}`);
console.log(`Tags: ${structuredResult.tags.join(", ")}`);
console.log(`Primary Objects: ${structuredResult.primary_objects.join(", ")}`);
console.log(`Scene Type: ${structuredResult.scene_type}`);


Raw JSON Response:
{"caption":"The image captures a bustling urban street scene during the daytime, likely in a major city such as New York, indicated by the yellow taxi cab. In the foreground and midground, several pedestrians are actively crossing a wide street at a crosswalk. On the right, a man with a beard, wearing a light blue shirt and khaki pants, walks briskly, holding a phone and wearing earbuds. Beside him, a woman pushes a stroller containing a small child, while another young child walks alongside the stroller. On the left side of the crosswalk, a man with a backpack, dressed in a dark jacket and jeans, stands near a yellow Toyota RAV4 taxi. The street is filled with various vehicles, including parked cars, SUVs, and a white commercial van, and more traffic is visible further down the street. The background is dominated by tall, densely packed commercial and residential buildings lining both sides of the street, some featuring prominent signs like 'Office' and large billbo

## Example 3: Batch Processing Multiple Images

Let's process multiple images and display the results.


In [16]:
// Sample images to process
const sampleImages = [
    "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.caption/car.jpg",
    "https://images.unsplash.com/photo-1506905925346-21bda4d32df4?w=400",  // Mountain landscape
    "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",  // Cat
    "https://images.unsplash.com/photo-1449034446853-66c86144b0ad?w=400",  // City buildings
];

interface BatchResult {
    imageUrl: string;
    caption: string;
    tags: string;
    primaryObjects: string;
    sceneType: string;
    error?: string;
}

async function processImagesBatch(imageUrls: string[]): Promise<BatchResult[]> {
    /**
     * Process multiple images and return structured results.
     */
    const results: BatchResult[] = [];
    
    for (const url of imageUrls) {
        try {
            console.log(`Processing: ${url}`);
            const result = await generateStructuredCaption(url);
            results.push({
                imageUrl: url,
                caption: result.caption,
                tags: result.tags.join(", "),
                primaryObjects: result.primary_objects.join(", "),
                sceneType: result.scene_type
            });
        } catch (error) {
            console.log(`Error processing ${url}: ${error}`);
            results.push({
                imageUrl: url,
                caption: `Error: ${error}`,
                tags: "",
                primaryObjects: "",
                sceneType: "",
                error: String(error)
            });
        }
    }
    
    return results;
}

// Process images
const batchResults = await processImagesBatch(sampleImages);

// Display results as a table
console.log("\n=== Batch Processing Results ===");
console.table(batchResults.map(r => ({
    url: r.imageUrl.substring(0, 50) + "...",
    caption: r.caption.substring(0, 60) + "...",
    tags: r.tags,
    sceneType: r.sceneType
})));


Processing: https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.caption/car.jpg
Raw JSON Response:
{"caption":"The image features a light blue or teal vintage Volkswagen Beetle, prominently parked on a cobblestone or paved street. The car is well-maintained, showcasing chrome bumpers and hubcaps. In the background, there is a building with a light yellow or ochre-colored stucco wall. This wall contains two distinct wooden entrances: one is a double door with an arched top, and the other is a single rectangular door framed in white. The scene is brightly lit, suggesting a sunny day, and the overall atmosphere is calm and nostalgic, reminiscent of a charming, older urban setting.","primary_objects":["light blue classic car (a Volkswagen Beetle)","building facade with a light yellow wall","wooden door","wooden window"],"scene_type":"Street","tags":["car","Volkswagen Beetle","classic car","vintage car","automobile","vehicle","turquoise car","mint green car","light blue c

## Example 4: Custom Caption Styles

You can customize the caption style by adjusting the prompt. Let's try different captioning approaches.


In [13]:
// Different caption styles
const captionStyles: Record<string, string> = {
    "Short & concise": "Generate a brief, one-sentence caption for this image.",
    "Detailed & descriptive": "Generate a detailed, descriptive caption for this image with at least 50 words.",
    "Creative & poetic": "Generate a creative and poetic caption for this image.",
    "SEO-optimized": "Generate an SEO-optimized caption for this image that includes relevant keywords.",
    "Accessibility focused": "Generate an accessibility-focused alt text for this image that would help visually impaired users understand the content."
};

console.log("Different Caption Styles for the Same Image:\n");
console.log("=".repeat(80));

for (const [styleName, prompt] of Object.entries(captionStyles)) {
    const captionResult = await generateCaption(imageUrl, prompt);
    console.log(`\n${styleName.toUpperCase()}:`);
    console.log(captionResult);
    console.log("-".repeat(80));
}


Different Caption Styles for the Same Image:


SHORT & CONCISE:
A family with a stroller crosses a busy New York City street, filled with yellow taxis and towering buildings.
--------------------------------------------------------------------------------

DETAILED & DESCRIPTIVE:
A bustling urban street scene unfolds, likely in New York City, depicting a moment of pedestrian activity and vehicular traffic. In the foreground, several individuals are crossing a wide street at a marked crosswalk. Prominently featured is a family unit, with two adults and a small child being pushed in a blue stroller, moving from right to left. A man with a backpack walks on the left, positioned near a bright yellow taxi cab. The street is lined with numerous tall buildings, characteristic of a metropolitan area, some displaying advertisements and signs like 'Office'. Various cars and vans are visible on the road, both parked and in motion, adding to the dynamic cityscape under what appears to be a clear s

## Example 5: Using the VLM Run Node.js SDK

You can also use the official VLM Run Node.js SDK which provides a convenient wrapper around the OpenAI client.


In [14]:
// Alternative: Using VLM Run SDK directly
// import { VlmRun } from "vlmrun";
// 
// const vlmClient = new VlmRun({
//     apiKey: VLMRUN_API_KEY,
//     baseURL: "https://agent.vlm.run/v1"
// });
// 
// // Use the agent completions interface
// const response = await vlmClient.agent.completions.create({
//     model: "vlm-agent-1",
//     messages: [
//         {
//             role: "user",
//             content: [
//                 { type: "text", text: "Describe this image" },
//                 { type: "image_url", image_url: { url: imageUrl } }
//             ]
//         }
//     ]
// });
// 
// console.log(response.choices[0].message.content);

console.log("💡 The VLM Run SDK provides a convenient wrapper around the OpenAI client.");
console.log("   Install it with: npm install vlmrun");


💡 The VLM Run SDK provides a convenient wrapper around the OpenAI client.
   Install it with: npm install vlmrun


## Best Practices & Tips

### Caption Quality
- Be specific in your prompts about the level of detail you need
- For accessibility, ask for alt-text specifically
- For SEO, ask for keyword-rich descriptions

### Performance
- Use batch processing for multiple images
- Consider caching captions to avoid reprocessing
- Use appropriate image sizes (doesn't need to be full resolution)

### TypeScript Benefits
- Use Zod schemas for type-safe structured outputs
- Leverage TypeScript's type inference for better IDE support
- Use async/await for cleaner asynchronous code

### Tag Categories
VLM Run supports various tag types:
- **Objects**: person, car, truck, bus, bicycle, motorcycle
- **Scenes**: street, building, park, forest, beach
- **Time**: morning, afternoon, evening, night
- **Weather**: sunny, cloudy, rainy, snowing


## Additional Resources

- [VLM Run Documentation](https://docs.vlm.run)
- [API Reference](https://docs.vlm.run/agents/capabilities/image/captioning)
- [VLM Run Node.js SDK](https://github.com/vlm-run/vlmrun-node-sdk)
- [More Examples](https://github.com/vlm-run/vlmrun-cookbook)
- [Discord Community](https://discord.gg/AMApC2UzVY)
